In [ ]:
import requests
import folium
import polyline
import json


In [ ]:

def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out


In [ ]:

    
def get_map(route, line=True):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    if line:
        folium.PolyLine(
            route['route'],
            weight=8,
            color='blue',
            opacity=0.6
        ).add_to(m)
    else:
        for idx, marker in enumerate(route['route']):
            folium.Marker(
            location=(marker[1], marker[0]),
            icon=folium.Icon(icon='play', color=route['route_colors'][idx])
        ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m


In [ ]:

pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = 19.8512,45.246,19.841270,45.253672

with open('route_visited.json', 'r') as j:
     r = json.loads(j.read())['arr']
     rt = r[0]
     rc = r[1]
test_route = {
    'route': rt,
    'start_point': (rt[0][1], rt[0][0]),
    'end_point': ((rt[-1][1], rt[-1][0])),
    'route_colors': rc
}
m=get_map(test_route, line=False)

In [ ]:
test_route

In [ ]:
m